# Predicting Cardiovascular Disease Interface


This is the user interface designed to predict the probability of cardiovascular disease based on various characteristics provided by the user, including:
- Age
- Height
- Weight
- Gender
- Systolic blood pressure
- Diastolic blood pressure
- Cholesterol levels
- Glucose levels
- Smoking status
- Alcohol intake
- Physical activity
  
The input for each characteristic is analyzed using an artificial neural network. The neural network is trained on a dataset available at https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset, achieving an accuracy of approximately 72%. It's important to note that this prediction is not a substitute for a comprehensive diagnosis by medical professionals, as it has limitations. For an accurate diagnosis of cardiovascular disease, please consult healthcare experts.


1. import the dataset - done
2. perform cleaning - done
3. create ANN model
4. prompt the user for input
5. check validity of input
6. show results of the user input

In [1]:
import pandas as pd
import ipywidgets as widgets

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
#need to take this out
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, ConfusionMatrixDisplay

from IPython.core.display import HTML
from IPython import display
from IPython.display import Javascript, display
from ipywidgets import HBox, Label, interact, IntText

In [2]:
cardio = pd.read_csv('./cardio_train.csv', sep=';')
cardio.head()

# # Converting data from days to years
# cardio['age'] = cardio['age'] / 365
    
# # Filtering out improbable data
# cardio = cardio[(cardio['ap_hi'] > 0 & (cardio['ap_lo'] > 0))]
# cardio = cardio[(cardio['age'] >= cardio['age'].quantile(0.05)) & (cardio['age'] <= cardio['age'].quantile(0.95)) &
#                  (cardio['height'] >= cardio['height'].quantile(0.05)) & (cardio['height'] <= cardio['height'].quantile(0.95))]

# # Calculate BMI (Body Mass Index)
# cardio['bmi'] = cardio['weight'] / ((cardio['height'] / 100) ** 2)

# # Filter out outliers in BMI
# cardio_filtered = cardio[(cardio['bmi'] >= cardio['bmi'].quantile(0.05)) & (cardio['bmi'] <= cardio['bmi'].quantile(0.95))]
# cardio['bmi'] = cardio_filtered['bmi']
# cardio = cardio.dropna()
# cols = list(cardio.columns)
# cols.remove('bmi')
# cols.insert(-1, 'bmi')
# cardio = cardio[cols]

# # Transforming columns
# numeric_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'bmi']
# cat_cols = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']
# preproc = ColumnTransformer(
#     transformers = [
#         ('standard_scaler', StandardScaler(), numeric_cols),
#         ('categorical_cols', OneHotEncoder(handle_unknown='ignore'), cat_cols)
#     ]
# )

# # Getting feature and response columns
# dataset = cardio[numeric_cols + cat_cols]
# result = cardio['cardio']

# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(dataset,
#                                                    result,
#                                                    test_size=0.2,
#                                                    random_state=1)

# # Artificial Neural Network
# pl_nn = Pipeline([
#     ('preprocessor', preproc),
#     ('nn', MLPClassifier()),
# ])
# pl_nn.fit(X_train, y_train)

# # Getting the accuracy of the training and testing set
# # training_prediction_nn = pl_nn.predict(X_train)
# # testing_prediction_nn = pl_nn.predict(X_test)
# # training_accuracy_nn = metrics.accuracy_score(training_prediction_nn, y_train)
# # testing_accuracy_nn = metrics.accuracy_score(testing_prediction_nn, y_test)
# # print(f'Training accuracy: {training_accuracy_nn}')
# # print(f'Testing accuracy: {testing_accuracy_nn}')

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
# style = {'description_width': 'initial'}

# @interact
# def display_form():
#     print("Please fill out the following form to assess your likelihood of developing Cardiovascular Disease. It is important to note that the results of this prediction are not entirely accurate, and it is highly recommended to consult with medical professionals for a more comprehensive evaluation.")

# @interact(birth_date=widgets.DatePicker(description='Birthday', style = style))
# def assess_birthday(birth_date):
#     print(f"Selected birthday: {birth_date}")

# @interact(height=widgets.IntSlider(value=136, min=0, max=272, description='Height in Centimeters', style = style))
# def assess_height(height):
#     print(f"Selected height: {height} centimeters")

# @interact(weight=widgets.FloatSlider(value=60, min=2, max=635, step=0.1, description='Weight in Kilograms', readout_format='.1f', style=style))
# def assess_weight(weight):
#     print(f"Selected weight: {weight} kilograms")

# @interact(Gender=widgets.Dropdown(options=[('Select', ''), ('Man', 'Man'), ('Woman', 'Woman')], style=style))
# def assess_Gender(Gender):
#     print(f"Selected Gender: {Gender}")

# def display_blood_pressure(systolic, diastolic):
#     print(f"Blood Pressure: {systolic} / {diastolic}")
# systolic = IntText(value=0, description='Systolic')
# diastolic = IntText(value=0, description='Diastolic')
# interact(display_blood_pressure, systolic=systolic, diastolic=diastolic)

# @interact(Cholesterol=widgets.Dropdown(options=[('Select', ''), ('Normal', 'Normal'), ('Above Normal', 'Above Normal'), ('Well Above Normal', 'Well Above Normal')], style = style))
# def assess_cholesterol(Cholesterol):
#     print(f"Selected Cholesterol: {Cholesterol}")

# @interact(Glucose=widgets.Dropdown(options=[('Select', ''), ('Normal', 'Normal'), ('Above Normal', 'Above Normal'), ('Well Above Normal', 'Well Above Normal')], style = style))
# def assess_Glucose(Glucose):
#     print(f"Selected Glucose: {Glucose}")

# @interact(Smoking=widgets.Dropdown(options=[('Select', ''), ('I have smoked', 'I have smoked'), ('I have never smoked', 'I have never smoked')], style = style))
# def assess_Smoking(Smoking):
#     print(f"Selected Smoking: {Smoking}")

# @interact(Alcohol=widgets.Dropdown(options=[('Select', ''), ('I consume alcohol', 'I consume alcohol'), ('I do not consume alcohol', 'I do not consume alcohol')], style = style))
# def assess_Alcohol(Alcohol):
#     print(f"Selected Alcohol: {Alcohol}")

# @interact(Physical=widgets.Dropdown(options=[('Select', ''), ('I engage in regular physical activity', 'I engage in regular physical activity'), ('I do not engage in regular physical activity', 'I do not engage in regular physical activity')], style = style))
# def assess_Physical(Physical):
#     print(f"Selected Physical: {Physical}")


interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(DatePicker(value=None, description='Birthday', step=1, style=DescriptionStyle(descriptio…

interactive(children=(IntSlider(value=136, description='Height in Centimeters', max=272, style=SliderStyle(des…

interactive(children=(FloatSlider(value=60.0, description='Weight in Kilograms', max=635.0, min=2.0, readout_f…

interactive(children=(Dropdown(description='Gender', options=(('Select', ''), ('Man', 'Man'), ('Woman', 'Woman…

...

interactive(children=(Dropdown(description='Cholesterol', options=(('Select', ''), ('Normal', 'Normal'), ('Abo…

interactive(children=(Dropdown(description='Glucose', options=(('Select', ''), ('Normal', 'Normal'), ('Above N…

interactive(children=(Dropdown(description='Smoking', options=(('Select', ''), ('I have smoked', 'I have smoke…

interactive(children=(Dropdown(description='Alcohol', options=(('Select', ''), ('I consume alcohol', 'I consum…

interactive(children=(Dropdown(description='Physical', options=(('Select', ''), ('I engage in regular physical…

In [7]:
@interact
def display_form():
    print("Please fill out the following form to assess your likelihood of developing Cardiovascular Disease. It is important to note that the results of this prediction are not entirely accurate, and it is highly recommended to consult with medical professionals for a more comprehensive evaluation.")

style = {'description_width': 'initial'}

def assess_form(birth_date, height, weight, Gender, systolic, diastolic, Cholesterol, Glucose, Smoking, Alcohol, Physical):
    print(f"Selected birthday: {birth_date}")
    print(f"Selected height: {height} centimeters")
    print(f"Selected weight: {weight} kilograms")
    print(f"Selected Gender: {Gender}")
    print(f"Blood Pressure: {systolic} / {diastolic}")
    print(f"Selected Cholesterol: {Cholesterol}")
    print(f"Selected Glucose: {Glucose}")
    print(f"Selected Smoking: {Smoking}")
    print(f"Selected Alcohol: {Alcohol}")
    print(f"Selected Physical: {Physical}")

birth_date_widget = widgets.DatePicker(description='Birthday', style=style)
height_widget = widgets.IntSlider(value=136, min=0, max=272, description='Height in Centimeters', style=style)
weight_widget = widgets.FloatSlider(value=60, min=2, max=635, step=0.1, description='Weight in Kilograms', readout_format='.1f', style=style)
Gender_widget = widgets.Dropdown(options=[('Select', ''), ('Man', 'Man'), ('Woman', 'Woman')], style=style)
systolic_widget = widgets.IntText(value=0, description='Systolic')
diastolic_widget = widgets.IntText(value=0, description='Diastolic')
Cholesterol_widget = widgets.Dropdown(options=[('Select', ''), ('Normal', 'Normal'), ('Above Normal', 'Above Normal'), ('Well Above Normal', 'Well Above Normal')], style=style)
Glucose_widget = widgets.Dropdown(options=[('Select', ''), ('Normal', 'Normal'), ('Above Normal', 'Above Normal'), ('Well Above Normal', 'Well Above Normal')], style=style)
Smoking_widget = widgets.Dropdown(options=[('Select', ''), ('I have smoked', 'I have smoked'), ('I have never smoked', 'I have never smoked')], style=style)
Alcohol_widget = widgets.Dropdown(options=[('Select', ''), ('I consume alcohol', 'I consume alcohol'), ('I do not consume alcohol', 'I do not consume alcohol')], style=style)
Physical_widget = widgets.Dropdown(options=[('Select', ''), ('I engage in regular physical activity', 'I engage in regular physical activity'), ('I do not engage in regular physical activity', 'I do not engage in regular physical activity')], style=style)
submit_button = widgets.Button(description='Submit', style=style)

def on_submit_button_clicked(button):
    assess_form(
        birth_date_widget.value, height_widget.value, weight_widget.value, Gender_widget.value,
        systolic_widget.value, diastolic_widget.value, Cholesterol_widget.value,
        Glucose_widget.value, Smoking_widget.value, Alcohol_widget.value, Physical_widget.value
    )

submit_button.on_click(on_submit_button_clicked)

interact(assess_form,
         birth_date=birth_date_widget,
         height=height_widget,
         weight=weight_widget,
         Gender=Gender_widget,
         systolic=systolic_widget,
         diastolic=diastolic_widget,
         Cholesterol=Cholesterol_widget,
         Glucose=Glucose_widget,
         Smoking=Smoking_widget,
         Alcohol=Alcohol_widget,
         Physical=Physical_widget)

display(submit_button)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(DatePicker(value=None, description='Birthday', step=1, style=DescriptionStyle(descriptio…

Button(description='Submit', style=ButtonStyle())

Selected birthday: None
Selected height: 136 centimeters
Selected weight: 60.0 kilograms
Selected Gender: 
Blood Pressure: 0 / 0
Selected Cholesterol: 
Selected Glucose: 
Selected Smoking: 
Selected Alcohol: 
Selected Physical: 
